# Imports

In [6]:
# Standard library imports
import os
import math
import nest_asyncio
nest_asyncio.apply()

# Third-party imports
import pandas as pd
import requests
import torch
from transformers import pipeline
import lmdeploy
from lmdeploy import GenerationConfig

# Local imports

In [7]:
# Check available GPUs
print(f"Number of GPUs available: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Number of GPUs available: 8
Device 0: Tesla V100-PCIE-32GB
Device 1: Tesla V100-PCIE-32GB
Device 2: Tesla V100-PCIE-32GB
Device 3: Tesla V100-PCIE-32GB
Device 4: Tesla V100-PCIE-32GB
Device 5: Tesla V100-PCIE-32GB
Device 6: Tesla V100-PCIE-32GB
Device 7: Tesla V100-PCIE-32GB


# Llama-3.1-8B

In [9]:
# Set device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

# Get model and move to GPU
pipe = lmdeploy.pipeline(
    'meta-llama/Meta-Llama-3.1-8B-Instruct'
).to(DEVICE)

# Set generation configuration
gen_config = GenerationConfig(temperature=0, max_new_tokens=1024)

Using device: cuda


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

RuntimeError: [TM][ERROR] CUDA runtime error: out of memory /lmdeploy/src/turbomind/utils/memory_utils.cu:32 


# Check if Paper's Have Exact Value from Chembl

In [ ]:
doi = '10.1016/j.bmcl.2010.08.086'
txt_path = f'/data/rbg/users/vincentf/data_uncertainty/c340_txt/{doi.replace("/", "_")}.txt'
with open(txt_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Few-shot prompt with examples of input-output format
response = pipe([
    f'''    
    You are tasked with summarizing data from the literature provided. The text will contain information about an IC50 assay 
    for a target: digestive enzyme Cytochrome P450 3A4 (CYP3A4), and we only care about CYP3A4 data, not other variants. 

    Your task is to summarize key experimental details like assay type, inhibitors, and conditions.
    
    The output should only include details present in the text; do not infer or add anything. 

    INPUT:
    {text}
    
    OUTPUT:
    '''
])

In [ ]:
print(response[0].text)

# Chunk Data and See if It Contains Assay Information

In [ ]:
# chunks no more than 1024 tokens
token = pipe.tokenizer("text")
print(token)